## Task 4: Generate adversarial examples

In [1]:
import wandb
import os
os.environ["WANDB_NOTEBOOK_NAME"] = "generate_adversarial_examples.ipynb"
wandb.login()
import pandas as pd
import torch
from torch import optim, nn
from tqdm import tqdm
import pickle
from models import run_pytorch
from data import get_datasets

wandb: Currently logged in as: zhipeng-he. Use `wandb login --relogin` to force relogin


In [2]:
import foolbox
import numpy as np
from scipy.stats import chi2
import eagerpy as ep

In [3]:
def metrics(adv_numpy, sample_numpy, adv_numpy_cont, train_mean, train_cov_matrix, threshold, std):

    def _calculate_mahalanobis_distance(x, mean, cov_matrix):
        # Calculate the Mahalanobis distance
        diff = x - mean
        inv_cov_matrix = np.linalg.inv(cov_matrix)
        md = np.sqrt(np.dot(np.dot(diff, inv_cov_matrix), diff.T))
        return md[0, 0] # return a scalar
    
    eps = 1e-8

    # sparsity
    l0_distance = np.count_nonzero(adv_numpy - sample_numpy)

    # proximity
    l1_distance = np.linalg.norm(adv_numpy - sample_numpy, ord=1)
    l2_distance = np.linalg.norm(adv_numpy - sample_numpy, ord=2)
    linf_distance = np.linalg.norm(adv_numpy - sample_numpy, ord=np.inf)

    # deviation
    if adv_numpy_cont is not None:
        md = _calculate_mahalanobis_distance(adv_numpy_cont, train_mean, train_cov_matrix)
        # Check if the Mahalanobis distance exceeds the threshold
        is_outlier = md > threshold
    else:
        md = 0
        is_outlier = 0

    # sensitivity: 
    if l0_distance == 0:
        sens = 0
    else:
        sens = (l1_distance / ((std + eps) * l0_distance)).mean()

    return {
        "L0 Distance": l0_distance,
        "L1 Distance": l1_distance,
        "L2 Distance": l2_distance,
        "Linf Distance": linf_distance,
        "Mahalanobis Distance": md,
        "Is Outlier": is_outlier,
        "Sensitivity": sens,
    }


In [4]:
ATTACK_LIST = {
    "L2CarliniWagner": foolbox.attacks.L2CarliniWagnerAttack(binary_search_steps=10, steps=10, stepsize=0.1),
    "L2DeepFool": foolbox.attacks.L2DeepFoolAttack(),
    "LinfDeepFool": foolbox.attacks.LinfDeepFoolAttack(),
    "L2FGSM": foolbox.attacks.L2FastGradientAttack(),
    "LinfFGSM": foolbox.attacks.LinfFastGradientAttack(),
    "L2PGD": foolbox.attacks.L2ProjectedGradientDescentAttack(),
    "LinfPGD": foolbox.attacks.LinfProjectedGradientDescentAttack(),
    "L2BIM": foolbox.attacks.L2BasicIterativeAttack(),
    "LinfBIM": foolbox.attacks.LinfBasicIterativeAttack(),
    "L2Gaussian": foolbox.attacks.L2AdditiveGaussianNoiseAttack(),
    "L2Uniform": foolbox.attacks.L2AdditiveUniformNoiseAttack(),
    "LinfUniform": foolbox.attacks.LinfAdditiveUniformNoiseAttack(),

}

In [5]:
def save_datapoints_as_npy(alg_name, model_name, dataset_name, epsilon, datapoints):

    path = f"./datapoints/{alg_name}_{dataset_name}/{model_name}"
    os.makedirs(path, exist_ok=True)
    for k in datapoints.keys():
            with open(f"{path}/{alg_name}_{dataset_name}_{model_name}_{epsilon}_{k}_arr.npy", 'wb') as f:
                np.save(f, datapoints[k]["arr"])
            with open(f"{path}/{alg_name}_{dataset_name}_{model_name}_{epsilon}_{k}_arr_adv.npy", 'wb') as f:
                np.save(f, datapoints[k]["arr_adv"])
    
    path_set = {
        "success_arr": f"{path}/{alg_name}_{dataset_name}_{model_name}_{epsilon}_success_arr.npy",
        "success_arr_adv": f"{path}/{alg_name}_{dataset_name}_{model_name}_{epsilon}_success_arr_adv.npy",
        "arr": f"{path}/{alg_name}_{dataset_name}_{model_name}_{epsilon}_arr.npy",
        "arr_adv": f"{path}/{alg_name}_{dataset_name}_{model_name}_{epsilon}_arr_adv.npy",
    }

    return path_set


In [6]:
def run_attack(model, attack_name, epsilons, X_test_tensor, y_test_tensor, X_train_tensor, num_continues, config, device):
    # Create a Foolbox model wrapper for the PyTorch model
    fmodel = foolbox.models.PyTorchModel(model, bounds=(0, 1))

    # Create an attack object (FGSM in this case)
    attack = ATTACK_LIST[attack_name]


    # Generate an adversarial example

    attack_success_rates = []
    average_distances = {
        "L0 Distance": [], "L1 Distance": [], "L2 Distance": [], "Linf Distance": [], "Mahalanobis Distance": [], "Sensitivity": []
    }
    outliner_rates = []
    path_sets = []

    X_train_numpy = X_train_tensor.numpy()

    if num_continues != 0:
        # for calculating mahalanobis distance, use numerical features only
        X_train_numpy_continues = X_train_tensor[:,:num_continues].numpy()
        train_mean = np.mean(X_train_numpy_continues, axis=0)
        train_cov_matrix = np.cov(X_train_numpy_continues.T)

        # Calculate the critical value for the Mahalanobis distance using chi-squared distribution
        alpha = 0.05
        degrees_of_freedom = len(train_mean)
        # Calculate the Chi-Square critical value at the given alpha and df
        chi_square_critical_value  = chi2.ppf(1 - alpha, df=degrees_of_freedom)
        threshold = np.sqrt(chi_square_critical_value)
        print(f"Threshold: {threshold}")
    else:
        X_train_numpy_continues = None
        train_mean = None
        train_cov_matrix = None
        threshold = None
        print("No numerical features, cannot calculate Mahalanobis distance")
    
    std = np.std(X_train_numpy, axis=0)
    # print(f"std: {std}")



    for epsilon in epsilons:
        # Initialize variables to keep track of success and the number of samples
        total_samples = len(X_test_tensor)
        successful_attacks = 0
        total_metrics = {
            "L0 Distance": 0.0, "L1 Distance": 0.0, "L2 Distance": 0.0,
            "Linf Distance": 0.0, "Mahalanobis Distance": 0.0,
            "Is Outlier": 0, "Sensitivity": 0.0
        }

        arr_list=[]
        arr_list_adv=[]

        success_arr_list=[]
        success_arr_list_adv=[]

        for sample_idx in tqdm(range(total_samples)):
            sample = X_test_tensor[sample_idx].to(device).unsqueeze(0)
            label = y_test_tensor[sample_idx].to(device).unsqueeze(0)

            _, advs, success = attack(fmodel, sample, label, epsilons=[epsilon])

            with torch.no_grad():
                model.eval()
                test_outputs = model(sample)
                adv_outputs = model(advs[0])
                _, predicted_classes = test_outputs.max(dim=1)
                _, adv_predicted_classes = adv_outputs.max(dim=1)

            adv_numpy = advs[0].cpu().numpy()
            # print(adv_numpy.shape)
            sample_numpy = sample.cpu().numpy()
            sample_numpy_cont = adv_numpy[:,:num_continues]
            # print(sample_numpy_cont.shape)

            metrics_dict = metrics(adv_numpy, sample_numpy, sample_numpy_cont, train_mean, train_cov_matrix, threshold, std)

            arr_list.append(sample_numpy)
            arr_list_adv.append(adv_numpy)

            if success:
                successful_attacks += 1

                for key, value in metrics_dict.items():
                    total_metrics[key] += value

                success_arr_list.append(sample_numpy)
                success_arr_list_adv.append(adv_numpy)

            # print(f"Epsilon = {epsilon}, Predicted class: {predicted_classes} | Adversarial example: {adv_predicted_classes} | Success: {success} | L2 distance: {np.linalg.norm(advs[0].cpu().numpy() - sample.cpu().numpy())}")

        arr = np.concatenate(arr_list, axis=0)
        arr_adv = np.concatenate(arr_list_adv, axis=0)
        success_arr = np.concatenate(success_arr_list, axis=0)
        success_arr_adv = np.concatenate(success_arr_list_adv, axis=0)

        datapoints = {
            "all": {
                "arr": arr,
                "arr_adv": arr_adv,
            },
            "success": {
                "arr": success_arr,
                "arr_adv": success_arr_adv,
            }
        }

        path_set = save_datapoints_as_npy(attack_name, config["model"], config["dataset"], epsilon, datapoints)

        if successful_attacks > 0:
            success_rate = successful_attacks / total_samples
            outliner_rate = total_metrics["Is Outlier"] / successful_attacks
            for key, value in total_metrics.items():
                if key in average_distances:
                    average_distances[key].append(value / successful_attacks)

        else:
            success_rate = 0.0
            outliner_rate = 0.0
            for key, value in total_metrics.items():
                if key in average_distances:
                    average_distances[key].append(0.0)
        
        attack_success_rates.append(success_rate)
        outliner_rates.append(outliner_rate)
        path_sets.append(path_set)
        
        print(f"Epsilon = {epsilon}")
        print(f"Success Rate: {success_rate * 100}%")
        for key, value in average_distances.items():
            print(f"Average {key} for Successful Attacks: {value[-1]}")
        print(f"Outlier Rate for Successful Attacks: {outliner_rate * 100}%\n")
        print("")

    return attack_success_rates, average_distances, outliner_rates, path_sets

Run Attack

In [7]:
wandb_run = False

In [8]:
# ### Only for Testing

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# # device = torch.device("cpu")


# for dataset_name in ["Adult"]: # "Adult", "Electricity", "Higgs", "KDDCup09_appetency", "Mushroom"
#     X_train, y_train, X_val, y_val, X_test, y_test, \
#         X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor, X_val_tensor, y_val_tensor, \
#         info = get_datasets.get_split_continues(dataset_name, device)
#     for model_name in ["MLP"]: # , "TabTransformer", "FTTransformer"
#         model, train_config = run_pytorch.model_config(model_name, X_train.shape[1], 2, [], info.numerical_cols, device)
#         train_config["dataset"] = f"{dataset_name}_continuous_only"
#         train_config["device"] = device

#         criterion = nn.CrossEntropyLoss()
#         optimizer = run_pytorch.build_optimizer(model, "adam", train_config["learning_rate"])
#         path = f"models/train/{model_name}/{dataset_name}_continuous_only/train_run-test.pt"
#         model.load_state_dict(torch.load(path))
        
#         # run_pytorch.test(model, (X_test_tensor, y_test_tensor), train_config, stage="train", wandb_run=wandb.run)

#         ## LinfFGSM
#         epsilons = np.arange(0.01, 0.15, 0.02)  # You can change the epsilon values
#         attack_config = {**train_config, "epsilons": epsilons}
#         attack_success_rates, average_distances, outliner_rates = attack_LinfFGSM(model, epsilons, X_test_tensor, y_test_tensor, X_train_tensor, len(info.numerical_cols), attack_config, device)

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")


for dataset_name in ["WineQuality-Red", "WineQuality-White"]: # "Adult", "Electricity", "Higgs", "KDDCup09_appetency", "Mushroom", "BankMarketing", "house_16H", "GermanCredit", "jm1", "BreastCancer", "WineQuality-White", "WineQuality-Red"
    X_train, y_train, X_val, y_val, X_test, y_test, \
        X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor, X_val_tensor, y_val_tensor, \
        info = get_datasets.get_split(dataset_name, device)
    # Considering size of Higgs dataset, we will use only 10k samples for adversarial attack.
    if dataset_name == "Higgs":
        X_test = X_test[:10000]
        y_test = y_test[:10000]
        X_test_tensor = X_test_tensor[:10000]
        y_test_tensor = y_test_tensor[:10000]


    for model_name in ["LogisticRegression", "MLP", "TabTransformer", "FTTransformer"]: # "LogisticRegression", "MLP", "TabTransformer", "FTTransformer"
        model, train_config = run_pytorch.model_config(model_name, X_train.shape[1], 2, info.num_categories_list, info.numerical_cols, device)
        train_config["dataset"] = dataset_name
        train_config["device"] = device

        criterion = nn.CrossEntropyLoss()
        optimizer = run_pytorch.build_optimizer(model, "adam", train_config["learning_rate"])
        path = f"models/train/{model_name}/{dataset_name}/train_run-test.pt"
        model.load_state_dict(torch.load(path))
        
        # run_pytorch.test(model, (X_test_tensor, y_test_tensor), train_config, stage="train", wandb_run=wandb.run)

        # "L2Gaussian", "L2Uniform", "LinfUniform", 
        # "LinfFGSM", "LinfPGD", "LinfBIM", "L2FGSM", "L2PGD", "L2BIM", 
        # "LinfDeepFool", "L2DeepFool", "L2CarliniWagner", 

        for attack_name in [ 
            "L2Gaussian", "L2Uniform", "LinfUniform", 
            "LinfFGSM", "LinfPGD", "LinfBIM", "L2FGSM", "L2PGD", "L2BIM", 
            "LinfDeepFool", "L2DeepFool", "L2CarliniWagner", 
        ]: 
            print(f"Running {attack_name} on {dataset_name} with {model_name}")

            epsilons = [
                0.0,
                0.01,
                0.03,
                0.05,
                0.1,
                0.3,
                0.5,
                1.0,
            ]
            attack_config = {**train_config, "epsilons": epsilons}
            attack_success_rates, average_distances, outliner_rates, path_sets = run_attack(model, attack_name, epsilons, X_test_tensor, y_test_tensor, X_train_tensor, len(info.numerical_cols), attack_config, device)

            
            results = {
                "model": model_name,
                "dataset": dataset_name,
                "attack": attack_name,
                "attack_config": attack_config,
                "result": {
                    "epsilons": epsilons,
                    "attack_success_rates": attack_success_rates,
                    "average_distances": average_distances,
                    "outliner_rates": outliner_rates,
                    "numpy_path": path_sets,
                }
            }

            # save python dictionary
            with open(f'results/{dataset_name}_{model_name}_{attack_name}.pickle', 'wb') as handle:
                pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)


/home/zippo/miniconda3/envs/tabular-benchmark/lib/python3.10/site-packages/foolbox/models/pytorch.py:36: UserWarning: The PyTorch model is in training mode and therefore might not be deterministic. Call the eval() method to set it in evaluation mode if this is not intended.
  warnings.warn(


Running L2Gaussian on WineQuality-Red with LogisticRegression
Threshold: 4.4356665308251575


100%|██████████| 320/320 [00:01<00:00, 283.11it/s]


Epsilon = 0.0
Success Rate: 27.8125%
Average L0 Distance for Successful Attacks: 0.0
Average L1 Distance for Successful Attacks: 0.0
Average L2 Distance for Successful Attacks: 0.0
Average Linf Distance for Successful Attacks: 0.0
Average Mahalanobis Distance for Successful Attacks: 2.846129352706799
Average Sensitivity for Successful Attacks: 0.0
Outlier Rate for Successful Attacks: 3.3707865168539324%




100%|██████████| 320/320 [00:00<00:00, 596.06it/s]


...

Epsilon = 0.3
Success Rate: 83.77551020408164%
Average L0 Distance for Successful Attacks: 10.940316686967114
Average L1 Distance for Successful Attacks: 0.0738040540624301
Average L2 Distance for Successful Attacks: 0.10649146952830865
Average Linf Distance for Successful Attacks: 0.2483302525425586
Average Mahalanobis Distance for Successful Attacks: 3.6951505099925774
Average Sensitivity for Successful Attacks: 0.07935297776281122
Outlier Rate for Successful Attacks: 20.584652862362972%




100%|██████████| 980/980 [07:44<00:00,  2.11it/s]


Epsilon = 0.5
Success Rate: 95.71428571428572%
Average L0 Distance for Successful Attacks: 10.947761194029852
Average L1 Distance for Successful Attacks: 0.09637963344707394
Average L2 Distance for Successful Attacks: 0.1438111438401849
Average Linf Distance for Successful Attacks: 0.3404384913096236
Average Mahalanobis Distance for Successful Attacks: 3.9974786753454086
Average Sensitivity for Successful Attacks: 0.10362589059337828
Outlier Rate for Successful Attacks: 27.931769722814497%




100%|██████████| 980/980 [07:44<00:00,  2.11it/s]

Epsilon = 1.0
Success Rate: 98.57142857142858%
Average L0 Distance for Successful Attacks: 10.94927536231884
Average L1 Distance for Successful Attacks: 0.10349994884176261
Average L2 Distance for Successful Attacks: 0.15791635469880613
Average Linf Distance for Successful Attacks: 0.3778084612166602
Average Mahalanobis Distance for Successful Attacks: 4.156128965693517
Average Sensitivity for Successful Attacks: 0.11128154340228877
Outlier Rate for Successful Attacks: 30.020703933747413%


